### Importing dependencies ###

In [1]:
import csv
import pandas as pd
import numpy as np
import os

## Reading in all data 

The data is read in including files with endigs '*.csv', '*.csv.gz', and '*.geojson'

### Input folder struct: ###

```plain_text
folder_path/
├── City1/
│   ├── file1.csv
│   ├── file2.csv
│   └── file3.csv
├── City2/
│   ├── file4.csv
│   └── file5.geojson
└── City3/
    └── file6.csv.gz
```

### Output structure: ###

- Keys: City names (subdirectory names under `folder_path`).
- Values: Lists of Pandas DataFrames corresponding to the files in the respective subfolder.


```python
data_dict = {
    "City1": [DataFrame_file1, DataFrame_file2, DataFrame_file3],   # Files from `City1` subfolder
    "City2": [DataFrame_file1, DataFrame_file2],                    # Files from `City2` subfolder
    "City3": [DataFrame_file1]                                      # Files from `City3` subfolder
}

In [4]:


### Read all CSV files in a folder into a dictionary of DataFrames for every city/region in the folder###
folder_path = 'C:/Users/nilsk/Dokumente/Machine Learning (MSc.)/1. Semester/Data Literacy/DataLit-InsideAirbnb/data/raw_data/berlin'

# Initialize the dictionary to store DataFrames
data_dict = {}

for root, dirs, files in os.walk(folder_path):
    # Get the name of the subdirectory
    subdirectory_name = os.path.basename(root)
    
    # Skip the root folder itself if needed (optional)
    if subdirectory_name == os.path.basename(folder_path):
        continue
    
    # Initialize a list for the current subdirectory
    data_dict[subdirectory_name] = []

    print(f"Processing folder: {root}")
    
    for file_name in files:
        if file_name.endswith('.csv') or file_name.endswith('.geojson') or file_name.endswith('.csv.gz'):
            file_path = os.path.join(root, file_name)
            
            # Read the file into a DataFrame
            if file_name.endswith('.geojson'):
                df = pd.read_json(file_path)  # Adjust based on the specific geojson handling
            else:
                df = pd.read_csv(file_path)
            
            # Append the DataFrame to the list for this subdirectory
            data_dict[subdirectory_name].append(df)
            
            print(f"Loaded {file_path} into {subdirectory_name}'s list of DataFrames")


Processing folder: C:/Users/nilsk/Dokumente/Machine Learning (MSc.)/1. Semester/Data Literacy/DataLit-InsideAirbnb/data/raw_data/berlin\18_09_2024
Loaded C:/Users/nilsk/Dokumente/Machine Learning (MSc.)/1. Semester/Data Literacy/DataLit-InsideAirbnb/data/raw_data/berlin\18_09_2024\calendar.csv into 18_09_2024's list of DataFrames
Loaded C:/Users/nilsk/Dokumente/Machine Learning (MSc.)/1. Semester/Data Literacy/DataLit-InsideAirbnb/data/raw_data/berlin\18_09_2024\listings.csv into 18_09_2024's list of DataFrames
Loaded C:/Users/nilsk/Dokumente/Machine Learning (MSc.)/1. Semester/Data Literacy/DataLit-InsideAirbnb/data/raw_data/berlin\18_09_2024\neighbourhoods.csv into 18_09_2024's list of DataFrames
Loaded C:/Users/nilsk/Dokumente/Machine Learning (MSc.)/1. Semester/Data Literacy/DataLit-InsideAirbnb/data/raw_data/berlin\18_09_2024\reviews.csv into 18_09_2024's list of DataFrames
Processing folder: C:/Users/nilsk/Dokumente/Machine Learning (MSc.)/1. Semester/Data Literacy/DataLit-Inside

In [22]:
data_dict["18_09_2024"][1].iloc[4]["host_about"]

'Christian from Berlin\n\nHi, I´m a 52 years old german guy from Berlin. I´m an IT project manager and I work for the german railway company "Deutsche Bahn".\nI like to travel, so I´ve already been to many places. I speak german, english, spanish and some french and portuguese.'

## Merging data 

Converting data from the current data_dict format which is a dict->list->df_per_file:

```python
data_dict = {
    "City1": [DataFrame_file1, DataFrame_file2, DataFrame_file3],   # Files from `City1` subfolder
    "City2": [DataFrame_file1, DataFrame_file2],                    # Files from `City2` subfolder
    "City3": [DataFrame_file1]                                      # Files from `City3` subfolder
}
```
Into a dict->df_all_files_per_city:

```python
data_dict = {
    "City1": Dataframe_city1,                           # Files from `City1` subfolder
    "City2": Dataframe_city2,                           # Files from `City2` subfolder
    "City3": Dataframe_city3                            # Files from `City3` subfolder
}
```

In [30]:
for city, df_list in data_dict.items():
    # Concatenate all DataFrames in the list for the current city
    merged_df = pd.concat(df_list, ignore_index=True)
    
    # Replace the list of DataFrames with the merged DataFrame
    data_dict[city] = merged_df

# Print the keys and the shape of the merged DataFrames to verify
for city, df in data_dict.items():
    print(f"{city}: {df.shape}")

amsterdam: (4426150, 85)


In [31]:
print(data_dict['amsterdam'].keys())

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

## Merging into single df 

Merging the whole folder of city names with files into a single df by adding a new coloumn 'city_name' for each row and merging all dfs

In [32]:
df_list_with_region = []

for city, df in data_dict.items():
    df.insert(0, 'region', city)
    df_list_with_region.append(df)

combined_df = pd.concat(df_list_with_region, ignore_index=True)


In [35]:
# Display the combined DataFrame
combined_df.head(5)

,region,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,listing_id,date,available,adjusted_price,type,features,reviewer_id,reviewer_name,comments,neighbourhood_group
0,amsterdam,6.624170e+06,https://www.airbnb.com/rooms/6624170,2.024091e+13,2024-09-05,previous scrape,"Warm, cozy sunlighted downtown appt",2 room appt. 1.8 km from central station with ...,NaN,https://a0.muscache.com/pictures/df91da10-f7d4...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,amsterdam,8.837071e+06,https://www.airbnb.com/rooms/8837071,2.024091e+13,2024-09-05,previous scrape,Cozy apartment in city center,Located in Amsterdam's sweet spot. A stone's t...,see the guide,https://a0.muscache.com/pictures/5fee12d4-61d0...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,amsterdam,7.161070e+05,https://www.airbnb.com/rooms/716107,2.024091e+13,2024-09-05,previous scrape,Loft style home nearby city centre,NaN,NaN,https://a0.muscache.com/pictures/9927048/b367a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,amsterdam,6.645388e+17,https://www.airbnb.com/rooms/664538756986273255,2.024091e+13,2024-09-06,previous scrape,Geweldige duurzame eco woonark op unieke plek!,This unique eco houseboat is located in the mo...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,amsterdam,8.191077e+06,https://www.airbnb.com/rooms/8191077,2.024091e+13,2024-09-05,previous scrape,Old bar apartment,This just renovated apartment for 6 persons is...,Our apartment is situated in the centre of Ams...,https://a0.muscache.com/pictures/miso/Hosting-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
